In [5]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os

In [7]:
batch_size = 32
num_classes = 10
epochs = 100
data_augmentation = True
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

In [9]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [11]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

In [12]:
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))


In [13]:
# RMSProp optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Train model using RMSProp
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255


In [14]:
## Variations of data augmentation ##

if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # Real time augmentation
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        zca_epsilon=1e-06,  # epsilon for ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        # randomly shift images horizontally (fraction of total width)
        width_shift_range=0.1,
        # randomly shift images vertically (fraction of total height)
        height_shift_range=0.1,
        shear_range=0.,  # set range for random shear
        zoom_range=0.,  # set range for random zoom
        channel_shift_range=0.,  # set range for random channel shifts
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        cval=0.,  # value used for fill_mode = "constant"
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False,  # randomly flip images
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

Using real-time data augmentation.


In [15]:
 model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        epochs=epochs,
                        validation_data=(x_test, y_test),workers=4,steps_per_epoch=len(x_test)/32)

# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

# Scores for model
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Epoch 1/100
313/312 [==============================] - 52s 167ms/step - loss: 2.1457 - acc: 0.2025 - val_loss: 1.9495 - val_acc: 0.3137
Epoch 2/100
313/312 [==============================] - 48s 154ms/step - loss: 1.8917 - acc: 0.3091 - val_loss: 1.7643 - val_acc: 0.3800
Epoch 3/100
313/312 [==============================] - 48s 154ms/step - loss: 1.7933 - acc: 0.3457 - val_loss: 1.6563 - val_acc: 0.4031
Epoch 4/100
313/312 [==============================] - 50s 160ms/step - loss: 1.7284 - acc: 0.3589 - val_loss: 1.6023 - val_acc: 0.4196
Epoch 5/100
313/312 [==============================] - 54s 172ms/step - loss: 1.6858 - acc: 0.3827 - val_loss: 1.5224 - val_acc: 0.4405
Epoch 6/100
313/312 [==============================] - 51s 162ms/step - loss: 1.6259 - acc: 0.4059 - val_loss: 1.5148 - val_acc: 0.4436
Epoch 7/100
313/312 [==============================] - 52s 166ms/step - loss: 1.6184 - acc: 0.4078 - val_loss: 1.5399 - val_acc: 0.4332
Epoch 8/100
313/312 [===========================

Epoch 61/100
313/312 [==============================] - 50s 161ms/step - loss: 1.0154 - acc: 0.6353 - val_loss: 0.8951 - val_acc: 0.6859
Epoch 62/100
313/312 [==============================] - 51s 164ms/step - loss: 1.0440 - acc: 0.6321 - val_loss: 0.8906 - val_acc: 0.6875
Epoch 63/100
313/312 [==============================] - 52s 165ms/step - loss: 1.0247 - acc: 0.6438 - val_loss: 0.9043 - val_acc: 0.6842
Epoch 64/100
313/312 [==============================] - 53s 169ms/step - loss: 1.0192 - acc: 0.6424 - val_loss: 0.8618 - val_acc: 0.7017
Epoch 65/100
313/312 [==============================] - 51s 163ms/step - loss: 1.0121 - acc: 0.6459 - val_loss: 0.8914 - val_acc: 0.6878
Epoch 66/100
313/312 [==============================] - 51s 163ms/step - loss: 1.0018 - acc: 0.6519 - val_loss: 0.9440 - val_acc: 0.6683
Epoch 67/100
313/312 [==============================] - 51s 163ms/step - loss: 1.0127 - acc: 0.6448 - val_loss: 0.8675 - val_acc: 0.6991
Epoch 68/100
313/312 [===================